In [1]:
!pip install twython
from twython import Twython
!pip install tweepy

In [2]:
import os
import pandas as pd
import tweepy as tw
import time
import csv
import re 
import string
from tweepy import *

In [3]:
BT = "AAAAAAAAAAAAAAAAAAAAAJtGNQEAAAAA1Iek7lCeheF1fwY%2B77IpIHFABe4%3DFD1TrELX97CXY41A2iCCLwqpu1DAEuVjgTTesOLaL02ziFe6wp"
AT = "1367195278479142914-MDa7G2nlf33gfG3WKPr7VPRqe6tNPL"
ATS = "3TWSLjbDkOTDbJQo2KaSxapYoUyfM3DjWzXMVVQG0nlT6"
APIKey = "XxU2vxvZnGgm9pZzHFxcpnMl2"
APIKeySecret = "9K8YIhKLxJ39yqnpXzdwPZSxFQ9Q3zVZXAeZVoCFdASrYua5Uw"
auth = tw.OAuthHandler(APIKey, APIKeySecret)
auth.set_access_token(AT, ATS)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
def reTweetCount(tweetId):
    try:
        retweets_list = api.retweets(tweetId)
        print('worked')
        count = 0
        for retweet in retweets_list: 
            count = count + 1
        return count
    except tw.TweepError:

      # if tweepy encounters an error, sleep for fifteen minutes..this will
      # help against API bans.
            print('sleep5m')
            time.sleep(60 * 5)
            reTweetCount(tweetId)
    except KeyboardInterrupt:
        return

In [5]:
df = pd.DataFrame({})
search_words = "#"#"-filter:retweets"



In [ ]:
tweets = tw.Cursor(api.search,
                      q=search_words,
                      lang="en",
                      result_type='popular',
                      until = '2021-06-25',
                      count = 200)

In [ ]:
for tweet in tweets.items(int(200)):
            count = reTweetCount(tweet.id)
            df1 = pd.DataFrame({
                'id': [tweet.id],
                'user name': [tweet.user.screen_name],
                'text': [tweet.text],
                'time stamps': [tweet.created_at],
                'retweets': [count]
            })
            frames = [df, df1]
            df = pd.concat(frames)
df.to_excel('tweetsFirst200.xlsx')

In [8]:
df2 = pd.read_excel(r'tweetsFirst200.xlsx')
df2

,Unnamed: 0,id,user name,text,time stamps,retweets
0,0,1408077456456646912,BIGHIT_MUSIC,&lt; Jimin @ Photo Booth &gt;\n\n🎵 https://t.c...,2021-06-24 15:00:01,17
1,0,1408077587344104960,BIGHIT_MUSIC,&lt; SUGA @ Photo Booth &gt;\n\n🎵 https://t.co...,2021-06-24 15:00:32,18
2,0,1408077526425947904,BIGHIT_MUSIC,&lt; RM @ Photo Booth &gt;\n\n🎵 https://t.co/a...,2021-06-24 15:00:17,20
3,0,1408078078664790016,NCTsmtown_DREAM,"BIG DREAMS, BIG THRILLS\n\nNCT DREAM The 1st A...",2021-06-24 15:02:29,15
4,0,1408077969495518976,NCTsmtown_DREAM,"BIG DREAMS, BIG THRILLS #CHENLE\n\nNCT DREAM T...",2021-06-24 15:02:03,17
...,...,...,...,...,...,...
195,0,1406960095326608896,pledis_17,SEVENTEEN(세븐틴) 'Ready to love' M/V BEHIND THE ...,2021-06-21 13:00:01,19
196,0,1406930143533167104,loonatheworld,"Concept Photo - ""Kim Lip""\n\nLOOПΔ 4th Mini Al...",2021-06-21 11:01:00,14
197,0,1406887601647369984,itvnews,Exclusive: Amazon is destroying millions of it...,2021-06-21 08:11:57,20
198,0,1406811877795979008,marktuan,HAPPY FATHERS DAY! https://t.co/rFgNrgmfZs,2021-06-21 03:11:03,17


In [ ]:
lastID = df2['id'].iloc[-1]
lastID

In [ ]:
fn = 'allTweets.xlsx'
df2.to_excel(fn)

In [6]:
def getLastTweetID(dataFrame):
    return dataFrame['id'].iloc[-1]

In [9]:
def getLast200Tweets(lastID):
    try:
        df1 = pd.DataFrame({})
        tweets = tw.Cursor(api.search,
                          q=search_words,
                          lang="en",
                          max_id=lastID,
                          result_type='popular',
                          until = '2021-06-25',
                          count = 200)
        for tweet in tweets.items(int(200)):
            count = reTweetCount(tweet.id)
            dfT = pd.DataFrame({
                    'id': [tweet.id],
                    'user name': [tweet.user.screen_name],
                    'text': [tweet.text],
                    'time stamps': [tweet.created_at],
                    'retweets': [count]
            })
            frames = [df1, dfT]
            df1 = pd.concat(frames)
        return df1
    except tw.TweepError:

      # if tweepy encounters an error, sleep for fifteen minutes..this will
      # help against API bans.

            time.sleep(60 * 5)
            getLast200Tweets(lastID)
    except KeyboardInterrupt:
        return 

In [ ]:
i = 0
while (i<50):
    dfT = getLast200Tweets(lastID)
    frames = [df2, dfT]
    df2 = pd.concat(frames)
    lastID = getLastTweetID(df2)
    i = i + 1
df2.to_excel(fn)    

In [ ]:
df2

In [10]:
def searchTweetsByTag(tag):
    try:
        df1 = pd.DataFrame({})
        tweets = tw.Cursor(api.search,
                          q=tag,
                          lang="en",
                          until = '2021-06-25',
                          count = 200)
        for tweet in tweets.items(int(200)):
           # count = reTweetCount(tweet.id)
            dfT = pd.DataFrame({
                    'id': [tweet.id],
                    'user name': [tweet.user.screen_name],
                    'text': [tweet.text],
                    'time stamps': [tweet.created_at],
                    'retweets': [0]
            })
            frames = [df1, dfT]
            df1 = pd.concat(frames)
        return df1
    except tw.TweepError:

      # if tweepy encounters an error, sleep for five minutes..this will
      # help against API bans.

            time.sleep(60 * 5)
            searchTweetsByTag(tag)
    except KeyboardInterrupt:
        return

In [11]:
ashtagList = ['#love','#instagood','#photooftheday','#fashion','#beautiful','#happy','#cute','#tbt',
              '#like4like','#followme','#picoftheday','#follow','#me','#selfie','#summer','#art','#instadaily',
             '#friends','#repost','#nature','#girl','#fun','#style','#smile','#food','#instalike','#likeforlike',
             '#family','#travel','#fitness','#igers','#tagsforlikes','#follow4follow','#nofilter','#life','#beauty',
             '#amazing','#instamood','#instagram','#photography']
ashtagList2 = ['#medicaid','#ico','#ethereum','#aca','#crypto','#crowdfunding','#giveaway','#contest',
               '#blackhistorymonth','#womenshistorymonth','#cryptocurrency','#womensday','#happybirthday',
               '#authentication','#win','#medicare','#internationalwomensday','#influencermarketing',
               '#opioid','#healthinsurance','#qa','#funny','#womeninstem','#iwd2019','#photography','#mondaymotivation',
               '#ootd','#vegan','#traveltuesday','#tbt','#euro2020','#corona','#regionales2021']

In [ ]:

for tag in ashtagList:
    fileName = tag + 'Npop.xlsx'
    tagDF = searchTweetsByTag(tag)
    tagDF.to_excel(fileName)

In [ ]:

for tag in ashtagList2:
    fileName = tag + 'Npop.xlsx'
    tagDF = searchTweetsByTag(tag)
    tagDF.to_excel(fileName)

In [ ]:
twFromAllTags = pd.DataFrame({})
for tag in ashtagList:
    fn = tag + '.xlsx'
    tDF = twFromAllTags
    nDF = pd.read_excel(fn)
    frames = [tDF,nDF]
    twFromAllTags = pd.concat(frames, ignore_index=True)
    
for tag in ashtagList2:
    fn = tag + '.xlsx'
    tDF = twFromAllTags
    nDF = pd.read_excel(fn)
    frames = [tDF,nDF]
    twFromAllTags = pd.concat(frames, ignore_index=True)
    
twFromAllTags

In [ ]:
twFromAllTags.info()

In [ ]:
idList = twFromAllTags['id']
flag = len(set(idList)) == len(idList)
flag

In [ ]:
twFromAllTagsNpop = pd.DataFrame({})
for tag in ashtagList:
    fn = tag + 'Npop.xlsx'
    tDF = twFromAllTagsNpop
    nDF = pd.read_excel(fn)
    frames = [tDF,nDF]
    twFromAllTagsNpop = pd.concat(frames, ignore_index=True)
    
for tag in ashtagList2:
    fn = tag + 'Npop.xlsx'
    tDF = twFromAllTagsNpop
    nDF = pd.read_excel(fn)
    frames = [tDF,nDF]
    twFromAllTagsNpop = pd.concat(frames, ignore_index=True)
    
twFromAllTagsNpop

In [ ]:
twFromAllTagsNpop.info()

In [ ]:
idList = twFromAllTagsNpop['id']
flag = len(set(idList)) == len(idList)
flag

In [12]:
list2NpopDF = pd.DataFrame({})
for tag in ashtagList2:
    fn = tag + 'Npop.xlsx'
    tDF = list2NpopDF
    nDF = pd.read_excel(fn)
    frames = [tDF,nDF]
    list2NpopDF = pd.concat(frames, ignore_index=True)


In [13]:
list2NpopDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6410 entries, 0 to 6409
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   6410 non-null   int64         
 1   id           6410 non-null   int64         
 2   user name    6410 non-null   object        
 3   text         6410 non-null   object        
 4   time stamps  6410 non-null   datetime64[ns]
 5   retweets     6410 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 300.6+ KB


In [26]:
for i in range (len(list2NpopDF)):
    list2NpopDF['retweets'].iloc[i] = reTweetCount(list2NpopDF['id'].iloc[i])
list2NpopDF.to_excel('list2NpopReTwCou.xlsx')

sleep5m


KeyboardInterrupt: 

In [27]:
list2NpopDF['retweets'].max()

0